In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
df = pd.read_csv('logbook_assignment1.csv', low_memory=False)


## Question 1


In [ ]:
# 1.1.1
# determine what are not propper dates
# Improper formatting/garbage
# Propper date is month/day/year
# Note this only works if we clean up the stuff df["date_feuled"] = pd.to_datetime(df["date_feuled"], format='%b %d %Y', errors='raise')

# Define a function to check if the date is valid
def is_valid_date(date_str):
    if pd.isna(date_str):
        return False
    try:
        pd.to_datetime(date_str, format='%b %d %Y', errors='raise')
        return True
        
    except (ValueError, TypeError):
        return False

# Apply the function to the 'date_fueled' column
# Creating a new column valid_date_feuled (this is the data set in memory not the actual csv file)
# this column is a binary column into trues and false

df['valid_date_fueled'] = df['date_fueled'].apply(is_valid_date)

# Calculate the percentage of invalid dates
invalid_date_percentage = (df['valid_date_fueled'].value_counts(normalize=True) * 100)[False]

print(f"Percentage of invalid 'date_fueled' entries: {invalid_date_percentage:.2f}%")


In [ ]:
# 1.1.2
df['date_fueled'] = df.apply(lambda row: row['date_captured'] if not is_valid_date(row['date_fueled']) and is_valid_date(row['date_captured']) else row['date_fueled'], axis=1)
df.head(11)


In [ ]:
#1.1.3
df['date_fueled'] = pd.to_datetime(df['date_fueled'], errors='coerce')
df['date_captured'] = pd.to_datetime(df['date_captured'], errors='coerce')
print(df.dtypes)
df.head(11)

In [ ]:
#1.1.4
# NOTE do we change both columns, and when they say remove do they mean "NaT" or remove the row?

df['date_fueled'] = df.apply(lambda row: pd.NaT if (row['date_fueled'].year < 2005 or row['date_fueled']>pd.Timestamp.now()) else row['date_fueled'], axis=1)
df['date_captured'] = df.apply(lambda row: pd.NaT if (row['date_captured'].year < 2005 or row['date_captured']>pd.Timestamp.now()) else row['date_captured'], axis=1)

df.head()


In [ ]:
# #1.1.5

# # Sample DataFrame creation for demonstration purposes
# # df = pd.read_csv('your_data.csv')  # Uncomment and use your actual data

# # Ensure 'date_fueled' is in datetime format
# df['date_fueled'] = pd.to_datetime(df['date_fueled'], errors='coerce')

# # Drop rows where 'date_fueled' is NaT (not-a-time)
# df_graphing = df.copy()
# df_graphing = df_graphing.dropna(subset=['date_fueled'])

# # Extract day, month, and year
# df_graphing['day'] = df_graphing['date_fueled'].dt.day
# df_graphing['month'] = df_graphing['date_fueled'].dt.month
# df_graphing['year'] = df_graphing['date_fueled'].dt.year

# # Plot settings
# plt.figure(figsize=(10, 15))

# # Plot for day
# plt.subplot(3, 1, 1)
# sns.histplot(df_graphing['day'], bins=31, discrete=True, kde=False, color='blue')
# plt.title('Distribution of Fueling Dates by Day')
# plt.xlabel('Day of Month')
# plt.ylabel('Count')
# plt.xticks(range(1, 32))  # Set x-ticks to show all days of the month

# # Plot for month
# plt.subplot(3, 1, 2)
# sns.histplot(df_graphing['month'], bins=12, discrete=True, kde=False, color='green')
# plt.title('Distribution of Fueling Dates by Month')
# plt.xlabel('Month')
# plt.ylabel('Count')
# plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)

# # Plot for year
# plt.subplot(3, 1, 3)
# sns.countplot(data=df_graphing, x='year', palette='Reds')
# plt.title('Distribution of Fueling Dates by Year')
# plt.xlabel('Year')
# plt.ylabel('Count')
# plt.xticks(rotation=45)

# plt.tight_layout()
# plt.show()

In [ ]:
#1.2.1

missing_gallons = df['gallons'].isna().mean() * 100
missing_miles = df['miles'].isna().mean() * 100
missing_odometer = df['odometer'].isna().mean() * 100

# Print out the percentages
print(f"Percentage of missing values in 'gallons': {missing_gallons:.2f}%")
print(f"Percentage of missing values in 'miles': {missing_miles:.2f}%")
print(f"Percentage of missing values in 'odometer': {missing_odometer:.2f}%")


In [ ]:
#1.2.3

def to_float(x):
    try:
        # Remove any non-numeric characters (except for the decimal point)
        cleaned = x.replace(',', '')
        return float(cleaned)
    except:
        return None  # Return None if conversion fail
    
df['miles'] = df['miles'].apply(lambda x: to_float(x) if isinstance(x, str) else x)
df['odometer'] = df['odometer'].apply(lambda x: to_float(x) if isinstance(x, str) else x)
df['gallons'] = df['gallons'].apply(lambda x: to_float(x) if isinstance(x, str) else x)


df.head(10)

In [ ]:
#1.2.2
df['mpg'] = df['mpg'].apply(lambda x: to_float(x) if isinstance(x, str) else x)


def fix_measurements(row):
    if(pd.isna(row['gallons'])):
        
        row['gallons'] = (1/row['mpg']) * row['miles']
        
    if(pd.isna(row['miles'])):
        
        row['miles'] = row['mpg']*row['gallons']
        
    if(pd.isna(row['mpg'])):
        
        row['mpg'] = row['miles']/ row['gallons']
    return row
            
df = df.apply(lambda row: fix_measurements(row),  axis=1)

df.head(100)

In [ ]:
#1.2.4

# # Set up the plotting environment
# plt.figure(figsize=(12, 10))

# df_temp = df.copy()
# # Plot distribution for 'odometer'
# plt.subplot(2, 2, 1)
# sns.histplot(df_temp['odometer'].dropna(), kde=True, color='skyblue')
# plt.title('Distribution of Odometer Readings')
# plt.xlabel('Odometer (miles)')
# plt.ylabel('Frequency')

# # Plot distribution for 'gallons'
# plt.subplot(2, 2, 2)
# sns.histplot(df_temp['gallons'].dropna(), kde=True, color='salmon')
# plt.title('Distribution of Gallons Refueled')
# plt.xlabel('Gallons')
# plt.ylabel('Frequency')

# # Plot distribution for 'mpg'
# plt.subplot(2, 2, 3)
# sns.histplot(df_temp['mpg'].dropna(), kde=True, color='lightgreen')
# plt.title('Distribution of MPG')
# plt.xlabel('Miles per Gallon (MPG)')
# plt.ylabel('Frequency')

# # Plot distribution for 'miles'
# plt.subplot(2, 2, 4)
# sns.histplot(df_temp['miles'].dropna(), kde=True, color='orange')
# plt.title('Distribution of Miles Driven')
# plt.xlabel('Miles')
# plt.ylabel('Frequency')

# # Adjust layout for better fit
# plt.tight_layout()
# plt.show()

In [ ]:
#1.2.5

# Ensure columns are floats
df['odometer'] = df['odometer'].astype(float)
df['gallons'] = df['gallons'].astype(float)
df['mpg'] = df['mpg'].astype(float)
df['miles'] = df['miles'].astype(float)

# Compute descriptive statistics
statistics = pd.DataFrame()

for column in ['odometer', 'gallons', 'mpg', 'miles']:
    stats = df[column].describe()
    mode = df[column].mode()[0]  # Get the most frequent value
    statistics[column] = pd.Series({
        'mean': stats['mean'],
        'std': stats['std'],
        'min': stats['min'],
        '25%': stats['25%'],
        '50%': stats['50%'],
        '75%': stats['75%'],
        'max': stats['max'],
        'most_frequent': mode
    })

print(statistics)

## Question 2


In [ ]:
#2.1 currency column
def get_symbol_value(price):
    if isinstance(price, str):
        pattern =  r'(\D*)[\d\,\.]+(\D*)'
        unit = re.match(pattern, price.strip()).groups()
        numeric_value = re.findall(r'[\d\,\.]+', price.strip())[0]
        return (unit[0] or unit[1]).strip(), numeric_value.lstrip('.')
    return pd.NA, 0.0

df['currency'] = df['total_spent'].apply(lambda row: get_symbol_value(row)[0])

In [ ]:
#2.2 float value total spend and cost per gallon
df['total_spent_float'] = df['total_spent'].apply(lambda row: get_symbol_value(row)[1])
df['total_spent_float'] = df['total_spent_float'].apply(lambda x: to_float(x) if isinstance(x, str) else x)

df['cost_per_gallon_float'] = df['cost_per_gallon'].apply(lambda row: get_symbol_value(row)[1])
df['cost_per_gallon_float'] = df['cost_per_gallon_float'].apply(lambda x: to_float(x) if isinstance(x, str) else x)

In [ ]:
#2.3 car make, model, year, user id
def get_user_info(url):
    pattern = r'https://www\.somewebsite\.com/([^/]+)/([^/]+)/(\d+)/(\d+)'
    match = re.match(pattern, url.strip())
    if match:
        return match.groups()
    
    pattern = r'https://www\.somewebsite\.com/([^/]+)/([^/]+)/(\d+)'
    match = re.match(pattern, url.strip())
    if match:
        if len(match[2]) == 4:
            return match.group(1), match.group(2), match.group(3), pd.NA
        else:
            return match.group(1), match.group(2), pd.NA, match.group(3)
    
    pattern = r'https://www\.somewebsite\.com/([^/]+)/([^/]+)'
    match = re.match(pattern, url.strip())
    if match:
        return match.group(1), match.group(2), pd.NA, pd.NA

df[['car_make', 'model', 'year', 'user_id']] = pd.DataFrame(df['user_url'].apply(get_user_info).tolist(), index=df.index)

In [ ]:
#part 2 q2
#litres filled - using UK gallons
def to_litre(x):
    try:
        return float(x) * 4.54609
    except (ValueError, TypeError):
        return 0.0

df['litres_filled'] = df['gallons'].apply(to_litre)

In [ ]:
#km driven
def to_km(x):
    try:
        return float(x) * 1.60934
    except(ValueError, TypeError):
        return 0.0

df['km_driven'] = df['miles'].apply(to_km)

In [ ]:
#litres per 100km
df['litres_per_100km'] = df.apply(lambda row: (row['litres_filled'] / row['km_driven']) * 100 if row['km_driven'] != 0 else 0.0, axis=1)

print(df[['litres_filled', 'km_driven', 'litres_per_100km']])

## Question 3


In [ ]:
#Q3.1

# Step 1: Define a function to extract the currency symbol
def extract_currency(entry):
    match = re.match(r'[^0-9.,]+', str(entry))
    return match.group(0) if match else None

# Step 2: Apply the function to the 'total_spent' column to create a 'currency' column
df['currency'] = df['total_spent'].apply(extract_currency)

# Step 3: Remove NaN values from the 'currency' column
df = df.dropna()

# Step 4: Group by 'currency' and count unique 'user_id's
unique_users_per_currency = df.groupby('currency')['user_id'].nunique()

# Step 5: Sort the Series by the number of unique users in descending order
unique_users_per_currency = unique_users_per_currency.sort_values(ascending=False)

# Step 6: Limit the number of currencies to top 20 for display
top_currencies = unique_users_per_currency.head(20)

# Step 7: Plot the results as a horizontal bar plot
top_currencies.plot(kind='barh', figsize=(10, 8), color='skyblue')
plt.title('Top 20 Currencies by Number of Unique Users')
plt.xlabel('Number of Unique Users')
plt.ylabel('Currency')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest values at the top
plt.show()

In [ ]:
#3.2 

# Step 1: Extract the day of the month from 'date_fueled'
df['day_of_month'] = df['date_fueled'].dt.day

# Step 2: Group by 'day_of_month' and count unique 'user_id's
unique_users_per_day = df.groupby('day_of_month')['user_id'].nunique()

# Step 3: Sort the days of the month to ensure the plot is in the correct order
unique_users_per_day = unique_users_per_day.sort_index()

# Step 4: Plot the results as a bar plot
unique_users_per_day.plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('Number of Unique Users per Day of the Month')
plt.xlabel('Day of the Month')
plt.ylabel('Number of Unique Users')
plt.xticks(rotation=45)
plt.show()

In [ ]:
#3.3
# Step 1: Filter for the last known vehicle age for each user
df_temp = df.copy()

df_filtered = df_temp.dropna(subset=['user_id', 'date_fueled'])

df_latest_age = df_filtered.loc[df_filtered.groupby('user_id')['date_fueled'].idxmax()]


vehicle_age_mapping = df_latest_age.set_index('user_id')['date_fueled']
df_temp['vehicle_age'] = df_temp['user_id'].map(vehicle_age_mapping)
# Step 2: Calculate the average vehicle age per country

df_temp['vehicle_age'] = df_temp['vehicle_age'].dt.year

# Convert 'year' to datetime
df_temp['year'] = pd.to_numeric(df_temp['year'])

# Perform the subtraction
df_temp['vehicle_age'] = df_temp['vehicle_age'] - df_temp['year']

df_filtered = df_temp.dropna(subset=['currency', 'vehicle_age'])

average_age_per_country = df_filtered.groupby('currency')['vehicle_age'].mean().sort_values()
df['vehicle_age'] = df_temp['vehicle_age']
# Step 3: Plot the average vehicle age per country
plt.figure(figsize=(10, 16))
average_age_per_country.plot(kind='barh', color='skyblue')
plt.title('Average Vehicle Age per Country (Based on Last Known Age)')
plt.xlabel('Average Vehicle Age (Years)')
plt.ylabel('Country')
plt.show()

# Question 4 #

### 4.1 ###


In [ ]:

# Question 4.1
# 4.1.1 Identify the top 5 currencies by number of transactions

def replace_negatives(df):
    for col in df.columns:
        # Check if the column is numeric
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].apply(lambda x: x if x >= 0 else np.nan)
    return df
df = replace_negatives(df)
df = df.dropna()

filtered_df = df.dropna().copy()
currency_counts = filtered_df['currency'].value_counts()
print(currency_counts)

top_5_currencies = currency_counts.head(5)
print(top_5_currencies)
top_5_currencies = filtered_df['currency'].value_counts().head(5).index



In [ ]:
print(df.columns.tolist())
df = df.drop(columns=['cost_per_gallon','total_spent','valid_date_fueled','day_of_month'])
print(df.columns.tolist())
# Delete : day of month

In [ ]:
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from sklearn.decomposition import PCA




def clean_iforest(df_in,top_5_currencies,columns):
    df_top_5 = df_in[df_in['currency'].isin(top_5_currencies)]
    temp_df = df_in.copy()
    for country in top_5_currencies:
        
        # Filter the DataFrame for the current currency
        country_df = df_top_5.loc[df_top_5['currency'] == country]
        country_df.dropna()
        numerical_df = country_df[columns].copy()
        numerical_df.dropna()
        iso_forest = IForest(contamination=0.3, random_state=42)
        # Initialize and fit IFOREST
        iso_forest.fit(numerical_df)

        # Step 3: Get the outlier predictions and decision function scores
        outlier_predictions = iso_forest.predict(numerical_df)
        outlier_scores = iso_forest.decision_function(numerical_df)

        # Step 4: Perform PCA for 2D visualization
        outlier_indices = numerical_df.index[outlier_predictions == 1]
        temp_df = temp_df.drop(index=outlier_indices)
        
    return temp_df

def clean_abod_chunking(df_in,top_5_currencies,columns):
    
    temp_df = df_in.copy()
    
    df_top_5 = df_in[df_in['currency'].isin(top_5_currencies)]
    for country in top_5_currencies:
        country_df = df_top_5.loc[df_top_5['currency'] == country]
        numerical_df = country_df[columns]
        chunk_size = 10000
        abod = ABOD()        
        for start in range(0, numerical_df.shape[0], chunk_size):
            end = min(start + chunk_size, numerical_df.shape[0])
            chunk = numerical_df.iloc[start:end]

            # Normalize the chunk

            # Fit ABOD to the chunk
            abod.fit(chunk)

            # Get outlier scores and predictions
            outlier_predictions = abod.predict(chunk)
            outlier_scores = abod.decision_function(chunk)

            # Store results
            outlier_indices = chunk.index[outlier_predictions == 1]
            temp_df = temp_df.drop(index=outlier_indices)
    return temp_df

def clean_IQR(df_in, columns):
    df_top_5 = df_in[df_in['currency'].isin(top_5_currencies)]
    temp_df = df_in.copy()
    
    temp_df.dropna()
    
    for country in top_5_currencies:
        # Filter the DataFrame for the current currency
        country_df = df_top_5.loc[df_top_5['currency'] == country]
        country_df.dropna()
        for column in columns:
            Q1 = float(country_df[column].quantile(0.25))
            Q3 = float(country_df[column].quantile(0.75))
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            # Filter the data based on the calculated bounds
            country_df = country_df[(country_df[column] >= lower_bound) & (country_df[column] <= upper_bound)]        
        # Update temp_df with the filtered country_df
        temp_df = pd.concat([temp_df[temp_df['currency'] != country], country_df], ignore_index=True)
    
    return temp_df      

def Debug_by_Currency(top_5_currencies_in,df_in, columns):
    
    # get all that are in top 5 currencies
    df_top_5 = df_in[df_in['currency'].isin(top_5_currencies_in)]
    # columns = ['odometer', 'gallons', 'cost_per_gallon_float', 'total_spent_float', 'mpg', 'miles']

    for country in top_5_currencies_in:
        df_selected = df_top_5[df_top_5['currency'] == country]
        print("Count = ", df_selected['odometer'].count())
        df_selected = df_selected[columns]
        statistics = {
            'mean': df_selected.mean(),
            'std_dev': df_selected.std(),
            'max': df_selected.max(),
            'min': df_selected.min(),
            'most_frequent': df_selected.mode().iloc[0],  # Mode might return multiple values; take the first one
            '25th_percentile': df_selected.quantile(0.25),
            '50th_percentile': df_selected.median(),  # same as the 50th percentile (second quartile)
            '75th_percentile': df_selected.quantile(0.75)
        }

        statistics_df = pd.DataFrame(statistics)
        statistics_df.T.plot(kind='bar', figsize=(18, 15), subplots=True, layout=(5, 2), sharex=False, sharey=False, legend=False)
        plt.suptitle('Summary Statistics for Each Column')
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()

columns = ['odometer', 'gallons', 'cost_per_gallon_float', 'total_spent_float', 'mpg', 'miles','vehicle_age','km_driven','litres_filled','litres_per_100km',]

shape_before = df[df['currency'].isin(top_5_currencies)]
# Drop negatives

# clean with isoforest
iso_forest = IForest(contamination=0.3, random_state=42)
temp_df = clean_iforest(df,top_5_currencies,columns)
# clean with iqr
temp_df = clean_IQR(temp_df,columns)

### Debugging ####
Debug_by_Currency(top_5_currencies,temp_df,columns)
##################
df_top_5_cleaned = temp_df[temp_df['currency'].isin(top_5_currencies)]

df = df_top_5_cleaned

print("Shape before:",shape_before.shape)
print("Shape after:",df_top_5_cleaned.shape)


### 4.2 ###


In [ ]:
#4.2.1


#dictionary mapping symbol to currency conversion
country_to_conversion_rate = {
    '$': 15.3823251591,
    '£': 21.04047696756128,
    # 'RON': 3.561016617460096,
    '€': 17.55691938247276,
    # 'NZ$': 10.463745769290119,
    # 'zł': 3.8716213023422656,
    # 'AU$': 11.086558311963374,
    # '₩': 0.012921057634335107,
    # 'Ft': 0.049228365311083745,
    # 'Nkr': 1.7562412564644414,
    # 'R$': 2.7788176867721246,
    'CA$': 12.251039269863115,
    # 'RUB': 0.20142764755848228,
    # '₪': 4.946457139420966,
    # 'Rp': 0.0010748848810212465,
    # 'RM': 3.6807291683227543,
    'R': 1,
    # 'Skr': 1.704365292918337,
    # 'AR$': 0.14811930102973106,
    # 'TL': 1.1373840215881519,
    # 'din': 0.14932035549118555,
    # 'Ikr': 0.1197449878590314,
    # '฿': 0.46375845505834795,
    # 'CN¥': 2.421384013882797,
    # 'HK$': 1.9758579374105392,
    # 'Dkr': 2.35941913518826,
    # '₡': 0.024163120005340204,
    # '₴': 0.5525242324882099,
    # 'IRR': 0.0003659723194811983,
    # 'S$': 11.413865075923685,
    # 'Rs': 0.20738528391312752,
    # 'BGN': 8.976710339074849,
    # '¥': 0.13468751141507065,
    # 'AED': 4.188516040599048,
    # 'MKD': 0.28529865688539807,
    # '₱': 0.299648334554393,
    # 'BD': 40.910439252925535,
    # 'CL$': 0.018765782257773908,
    # 'EC$': 5.696443586124306,
    # '$U': 0.34485460284367786,
    # 'LE': 0.9814145154038807,
    # 'MXN': 0.7575695258543906,
    # 'CO$': 0.003838655875850146,
    # 'CFPF': 0.14712698442566022,
    # 'Kč': 0.7163508871733955,
    # 'kn': 0.4284,
    # 'JD': 21.695804173624825,
    # 'FCFA': 0.026765351055744145,
    # 'NT$': 0.5578116582862289,
    # 'BN$': 11.413865075923685,
    # 'RD$': 0.26613310406243895,
    # 'S/': 3.9724605473060772,
    # 'J$': 0.09953868028188678,
    # 'PKRs': 0.0872018831043759,
    # 'Ksh': 0.1355335469686133,
    # 'GY$': 0.07327442038835098,
    # 'KM': 8.976710339074849,
    # 'Fr': 16.82920454761222,
    # 'CFA': 0.026765351055744145,
    # 'MDL': 0.855142786969332,
    # 'MMK': 0.00867419550703364,
    # 'N$': 1,
    # '₦': 0.0371310997600941,
    # 'ZWL': 0.042504352470571986,
    # 'USh': 0.004363080796638006,
    # 'MURs': 0.34847827721159913,
    # 'BYN': 5.985339011770494,
    # 'TT$': 2.2604224913939928,
    # 'KD': 50.83663577953889,
    # 'B/': 15.3823251591,
    # 'Bs': 3.322801167298912,
    # 'SLRs': 0.0759050017757583,
    # 'SR': 4.10195337576,
    # 'GTQ': 1.9954878205245605,
    # 'QR': 4.225913505247253,
    # 'Br': 0.31075432884231613,
    # 'C$': 0.43446368059227175,
    # 'BWP': 1.3308681017927695,
    # 'GH₵': 2.5391667740610786,
    # 'Tk': 0.18105372943303144,
    # 'SR$': 0.7164702146188335,
    # 'man': 9.048579453765345,
    # 'KHR': 0.003781842839877822,
    # 'BZ$': 7.6336903406485055,
    # 'Bds$': 7.69116257955,
    # 'Kz': 0.028725140452584053,
    # 'NPRs': 0.12955507350499923,
    # 'OMR': 40.00604722782835,
    # '₲': 0.002200965440971102,
    # 'LB£': 0.01020386411880597,
    # 'HNL': 0.6291915004092598,
    # 'AMD': 0.03208508408629746,
    # 'GEL': 4.998274582533776,
    # '₮': 0.005387418325120314,
    # 'MTn': 0.24148392508995453,
    # 'ZMW': 0.8935661742855417,
    # 'SH£': 21.04047696756128,
    # '₫': 0.0006761174978246287,
    # 'RWF': 0.015230903505700206,
    # 'MGA': 0.003877790661202371,
    # 'DT': 5.35677928998579,
    # 'VT': 0.13757754864171776,
    # 'MOP$': 1.9183086770976108,
    # 'TMT': 4.397624751569983,
    # 'SY£': 0.006122241970343508,
    # 'BS$': 40.910439252925535,
    # 'Af': 0.14670489360555453,
    # 'ALL': 0.14366899895467927,
    # 'DA': 0.10968235620351982,
    # 'TSh': 0.006671580489055182,
    # 'KY$': 18.75881679560058,
    # 'NAf': 8.605496369603088,
    # 'KZT': 0.035345307265764285,
    # 'YR': 0.06146636289731339,
    # 'MAD': 1.6664862724670706,
    # 'CV$': 0.15921755130563853,
    # 'IQD': 0.01053831002866959,
    # 'CU$': 15.3823251591,
    # 'KGS': 0.18143663176637523,
    # 'L$': 0.10305110712513678
}
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⡀⣀⠀⠀⠀⠀⠀⢀⡀⢀⣀⡀⠀⠀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢻⡝⣆⠀⠀⠀⠀⠈⣻⣿⣿⠟⠛⣿⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢿⡙⣦⢀⣤⣶⣶⣿⣻⡟⠀⣀⣿⣿⠃⣀⡤⠖⠒⠛⠒⠲⢤⡤⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣹⣾⣿⣿⠿⠛⠛⠛⠛⠛⠿⣏⡽⠛⠁⠀⠀⠀⠀⠀⠀⠀⠻⣞⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣠⣶⡟⠛⠛⠛⠛⢉⡵⠋⠀⠀⠀⠀⠀⠀⣀⡀⠀⠙⢦⠀⣀⣠⠤⠖⠒⠚⠉⠉⣉⣉⣙⣶⣤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣴⡾⠛⠉⠀⣧⠀⠀⠒⢠⠏⠀⠀⠀⣀⡴⠚⠉⠉⠉⠉⠉⠙⠚⣿⣍⡀⠀⢀⣠⣴⣾⠿⠛⠓⢋⣉⣙⣿⣷⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⢀⣤⡶⠛⠁⠀⣀⣀⠀⢸⡆⠀⢠⠏⠀⠀⢠⠞⠁⠀⠀⠀⢀⣠⣴⣾⡿⠭⠭⠥⢽⣷⡿⠛⢉⣠⠴⣶⣿⣿⣿⣦⡀⠀⣹⠀⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠘⣿⠀⠀⢠⡞⠁⠈⣷⢸⡇⠀⡽⠀⠀⠀⠀⠀⠀⠀⣠⢞⡿⠚⠉⠀⢀⣠⣤⡖⠚⠋⢳⠖⠉⠁⢰⣿⣿⣿⣭⣿⠿⣾⠏⠀⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⢸⡇⠀⢸⡄⠀⢠⡟⠀⢷⡀⡇⠀⠀⠀⠀⢀⣴⣺⠵⠋⢀⣠⠴⣺⣟⣿⡿⢿⣶⣤⣼⡤⠤⠤⠼⠛⠋⠉⠁⢀⡼⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠘⣧⠀⠀⠙⠒⠋⠀⠀⠈⣇⠀⠀⠀⠀⠀⠻⠿⠶⠶⣞⣉⡀⠀⣿⣿⡿⠿⢟⣏⡵⠟⠀⣀⠀⠀⢀⣀⣤⠴⢯⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠀⠹⣆⠀⠀⠀⢀⡀⠀⠀⣿⠀⠀⠀⠀⠀⠀⠀⠀⠈⠓⠒⠛⠛⠓⠓⠂⢉⡭⠟⠀⠀⠀⠉⠉⠙⠋⠁⠀⠀⠀⠈⠳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⢻⠀⠀⣾⣿⡇⠀⠀⢿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⠖⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣽⣄⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⡇⠀⣹⡽⠶⣄⠀⠸⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣤⠶⢋⣿⠟⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⣧⡾⠋⠀⠀⢹⡄⠀⣷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⣀⣀⣀⣀⣀⣀⣤⣤⣤⠴⠶⢚⣛⣭⣴⠶⢿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⡟⠁⠀⠀⠀⢸⡟⣦⢿⠀⠀⠀⠀⠀⠀⠀⣠⡶⠟⢋⣉⣉⣉⣉⣉⣉⣥⣤⣤⡴⠶⠶⠞⠛⢋⣉⣡⣬⣿⠟⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⢿⣤⠤⣤⣀⠀⣾⠁⡿⠸⡇⠀⠀⠀⠀⠀⠀⣟⣀⠛⢋⣉⣩⣭⣥⣤⣵⣤⡴⠴⠾⠒⠒⠛⠋⠉⢉⡤⠚⣿⢷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⡏⣶⣯⠬⠾⣷⠿⠃⢰⡇⠀⠀⠀⠀⠰⣄⠙⠛⠉⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣴⠏⠀⠀⠈⢷⡝⣄⠀⠀⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⠀⢠⡟⣰⢿⡇⠀⠀⠀⠀⠀⢸⡆⠀⠀⣀⠀⠀⠈⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⡀⠀⠀⠀⠀⠙⢮⠳⣄⠀⠀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⠀⢀⣿⡽⠃⢸⡇⠀⢀⣤⣦⠀⠀⢷⠀⠀⠈⠳⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣾⠇⠀⠀⠀⠀⠀⠈⢷⡝⢦⡀⠀⠀⠀⠀
# ⠀⠀⠀⠀⠀⠀⢀⣾⣱⠃⠀⢈⣧⠀⠸⠿⠋⠀⠀⢸⠀⠀⠐⠀⠀⠀⠀⠀⠀⢀⣀⣀⣀⣀⣀⣤⡤⠤⠤⣤⠶⠚⠋⠉⡇⠀⠀⠀⠀⠀⠀⠀⠀⢹⡌⣟⢳⣖⢺⠃
# ⠀⠀⠀⠀⠀⢀⣾⢳⠏⠀⠀⠘⢿⣄⣤⣀⣀⣀⣀⣻⣀⣀⡤⠴⠶⠶⢴⣖⠛⠉⣩⠟⢻⣿⣿⠈⠓⣦⢞⣁⣀⣠⣤⣤⡇⠀⠀⠀⠀⠀⠀⠀⠀⠸⡇⢸⢢⣿⠉⠀
# ⣾⡛⠶⡤⠴⢾⣣⡏⠀⠀⠀⠀⢈⡇⠀⠀⠀⠀⠀⠉⡏⠁⠀⠀⠀⠀⢀⣈⣹⣾⣥⣤⡾⠃⢹⣷⣿⣿⣿⣿⣿⣿⣿⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⣧⢀⢸⡼⣇⠀
# ⠉⠛⢻⡆⢰⡏⠙⡇⠀⠀⠀⠀⢸⣿⣿⣶⣶⣶⣶⣶⡷⢶⣶⣾⣿⣿⣿⣿⠹⣿⣿⣿⣧⣀⣰⡿⠟⠋⠈⠉⠁⠀⠀⣸⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⢸⢈⣧⢸⠀
# ⠀⠀⣾⣀⣌⣃⣸⠇⠀⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣿⣧⣼⣿⣿⣿⣛⣋⣉⣄⣙⣡⣠⣨⣽⣯⣀⣀⣀⣀⣀⣠⡤⠴⠟⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠿⠞⠉⠉⠀⠀⠀

# Assuming the DataFrame is named df

# Convert 'date_fueled' to datetime if not already done
df['date_fueled'] = pd.to_datetime(df['date_fueled'], errors='coerce')

# Filter for January 2022
jan_2022_df = df[(df['date_fueled'].dt.year == 2022) & (df['date_fueled'].dt.month == 1)].copy()


# Convert cost_per_gallon to cost_per_liter in ZAR
def convert_to_zar(row):
    currency = row['currency']
    if currency in country_to_conversion_rate:
        conversion_rate = country_to_conversion_rate[currency]
        cost_per_gallon_zar = row['cost_per_gallon_float'] * conversion_rate
        cost_per_liter_zar = cost_per_gallon_zar / 3.78541  # Gallon to liter conversion
        return cost_per_liter_zar
    else:
        return None

# Apply conversion and add new column
jan_2022_df['cost_per_liter_zar'] = jan_2022_df.apply(convert_to_zar, axis=1)

# Drop rows with None in 'cost_per_liter_zar'
jan_2022_df = jan_2022_df.dropna(subset=['cost_per_liter_zar'])

# Print results or save to file
# jan_2022_df[['date_fueled', 'currency', 'cost_per_liter_zar']].head(30)

# Calculate the mean cost per liter per currency
mean_cost_per_liter_per_currency = jan_2022_df.groupby('currency')['cost_per_liter_zar'].mean()



# Plot mean cost per liter per currency
plt.figure(figsize=(20, 8))
mean_cost_per_liter_per_currency.plot(kind='bar', color='skyblue')

plt.xlabel('Currency')
plt.ylabel('Mean Cost per Liter (ZAR)')
plt.title('Mean Cost per Liter in ZAR for Each Currency (January 2022)')
plt.xticks(rotation=90)  # Rotate x-axis labels to vertical
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Show plot
plt.show()





In [ ]:
df_temp = df.copy()

# Assuming your dataframe is called 'df'
df_temp = df_temp.sort_values(['user_id', 'date_fueled'])

# Calculate the difference between consecutive odometer readings for each user
df_temp['odometer_diff'] = df_temp.groupby('user_id')['odometer'].diff()

# Calculate the average difference for each user
avg_diff = df_temp.groupby('user_id')['odometer_diff'].mean()

# Merge the average difference back to the main dataframe
df_temp = df_temp.merge(avg_diff.rename('avg_odometer_diff'), on='user_id')

# Identify potential missed fill-ups
df_temp['potential_missed_fillup'] = df_temp['odometer_diff'] > (2 * df_temp['avg_odometer_diff'])

# Count the number of potential missed fill-ups
missed_fillups_count = df_temp['potential_missed_fillup'].sum()

print(f"Estimated number of missed fill-ups: {missed_fillups_count}")

In [ ]:
# 4.2.3


# Sort by user and then by odometer reading
df = df.sort_values(by=['user_url', 'odometer'])

# Calculate distance per fill-up
df['distance_per_fill'] = df.groupby('user_url')['odometer'].diff()

# Drop NaN values (first fill-up for each user where we can't calculate distance)
df = df.dropna(subset=['distance_per_fill'])

# Group by country and calculate the average distance per tank
average_distance_per_tank = df.groupby('currency')['distance_per_fill'].mean().reset_index()

# Rename the columns for clarity
average_distance_per_tank.columns = ['currency', 'avg_distance_per_tank_km']

# Identify the country with the largest average distance
max_distance_country = average_distance_per_tank.loc[average_distance_per_tank['avg_distance_per_tank_km'].idxmax()]

# Plot the average distance per tank per country
plt.figure(figsize=(20, 8))
plt.bar(average_distance_per_tank['currency'], average_distance_per_tank['avg_distance_per_tank_km'])
plt.xlabel('Country')
plt.ylabel('Average Distance per Tank (km)')
plt.title('Average Distance per Tank per Country')
plt.xticks(rotation=90)  # Rotate x-axis labels to vertical
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()



In [ ]:
#4.2.4

# # Convert the 'year' column to numeric, forcing errors to NaN if any non-numeric values are present
# df['year'] = pd.to_numeric(df['year'], errors='coerce')


# df['vehicle_age'] = df['date_fueled'].dt.year - df['year']

# Group by vehicle age and calculate the average distance per fill-up
avg_distance_by_age = df.groupby('vehicle_age')['distance_per_fill'].mean().reset_index()

# Rename the columns for clarity
avg_distance_by_age.columns = ['vehicle_age', 'avg_distance_per_fill_km']

# Plot the average distance per fill-up by vehicle age
plt.figure(figsize=(10, 6))
plt.plot(avg_distance_by_age['vehicle_age'], avg_distance_by_age['avg_distance_per_fill_km'], marker='o')
plt.xlabel('Vehicle Age (years)')
plt.ylabel('Average Distance per Fill-Up (km)')
plt.title('Average Distance per Fill-Up vs Vehicle Age')
plt.grid(True)
plt.show()





In [ ]:
df.columns

In [ ]:
# 4.2.5

# Filter for vehicles in South Africa (currency set to R)
sa_vehicles_df = df[df['currency'] == 'R'].copy()

# Identify the top 5 most popular vehicles based on their 'car_make' and 'model'
top_vehicles = sa_vehicles_df.groupby(['car_make', 'model']).size().nlargest(5).index

# Filter the DataFrame for only the top 5 vehicles
top_vehicles_df = sa_vehicles_df[sa_vehicles_df.set_index(['car_make', 'model']).index.isin(top_vehicles)].copy()

# Convert mpg to km/l
# top_vehicles_df['km_per_l'] = top_vehicles_df['mpg'] * 0.425144

# Calculate the average fuel efficiency for each of the top 5 vehicles
fuel_efficiency = top_vehicles_df.groupby(['car_make', 'model'])['litres_per_100km'].mean().reset_index()

# Sort the result by fuel efficiency in descending order
fuel_efficiency = fuel_efficiency.sort_values(by='litres_per_100km', ascending=False)

# Display the results
fuel_efficiency




In [ ]:
#4.2.6

# Ensure the necessary columns are numeric and convert mpg to km/l
# df['km_per_l'] = df['mpg'] * 0.425144

# Group by currency and vehicle and calculate the average fuel efficiency
average_fuel_efficiency = df.groupby(['currency', 'car_make', 'model'])['litres_per_100km'].mean().reset_index()

# For each currency, sort the vehicles by fuel efficiency and select the top 5
top_5_per_country = average_fuel_efficiency.sort_values(by=['currency', 'litres_per_100km'], ascending=[True, False])
top_5_per_country = top_5_per_country.groupby('currency').head(5)

# Display the results
top_5_per_country

In [ ]:
df.columns

In [ ]:
#4.2.7


# 1. Filter for Canadian vehicles
df_canada = df[df['currency'] == 'CA$']

# 2. Identify top 5 Canadian vehicles
top_5_vehicles = df_canada.groupby(['car_make', 'model']).size().nlargest(5).index.tolist()

# 3. Determine the season for each entry
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

df_canada['season'] = df_canada['date_fueled'].apply(get_season)

# 4. Calculate average fuel efficiency for each vehicle in each season
df_top5 = df_canada[df_canada.set_index(['car_make', 'model']).index.isin(top_5_vehicles)]
avg_efficiency = df_top5.groupby(['car_make', 'model', 'season'])['litres_per_100km'].mean().reset_index()

# 5. Plot the results
plt.figure(figsize=(12, 6))
sns.barplot(x='season', y='litres_per_100km', hue='car_make', data=avg_efficiency)
plt.title('Fuel Efficiency of Top 5 Canadian Vehicles by Season')
plt.xlabel('Season')
plt.ylabel('Fuel Efficiency (L/100km)')
plt.legend(title='Vehicle Make/Model', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Optional: Print the data
print(avg_efficiency)

In [ ]:
#4.2.8

# Select relevant numerical features
features = [ 'litres_per_100km', 'vehicle_age', 'distance_per_fill', 'odometer', 'total_spent_float', 'litres_filled', 'km_driven']

# Create a correlation matrix
corr_matrix = df[features].corr()

# Create a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Heatmap of Fuel Efficiency and Related Features')
plt.tight_layout()
plt.show()

# Print the correlation values for fuel efficiency metrics
print("\nCorrelations with litres_per_100km:")
print(corr_matrix['litres_per_100km'].sort_values(ascending=False))


In [ ]:
#4.2.9

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# Define features
features = ['vehicle_age', 'distance_per_fill', 'odometer', 'total_spent_float', 
            'litres_filled', 'km_driven', 'car_make', 'model', 'currency']

# Create a copy of the dataframe with selected features
df_model = df[features + ['litres_per_100km']].copy()

# Encode categorical variables
le = LabelEncoder()
for col in ['car_make', 'model', 'currency']:
    df_model[col] = le.fit_transform(df_model[col].astype(str))

# Split features and target
X = df_model[features]
y = df_model['litres_per_100km']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Get feature importances
importances = rf_model.feature_importances_
feature_imp = pd.DataFrame({'feature': features, 'importance': importances})
feature_imp = feature_imp.sort_values('importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(12, 6))
plt.bar(feature_imp['feature'], feature_imp['importance'])
plt.xticks(rotation=45, ha='right')
plt.title('Feature Importance for Predicting Fuel Efficiency (litres_per_100km)')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.tight_layout()
plt.show()

# Print feature importances
print(feature_imp)

# Calculate and print the difference between each feature's importance
feature_imp['importance_diff'] = feature_imp['importance'].diff().abs()
print("\nDifference in importance between adjacent features:")
print(feature_imp[['feature', 'importance', 'importance_diff']])


## 4.3 ##


In [ ]:
import matplotlib.colors as mcolors

# Filter for South African drivers
sa_drivers_df = df.loc[df['currency'] == 'R'] 

# Normalize the price values to range from 0 to 1 for colormap scaling
norm = plt.Normalize(sa_drivers_df['cost_per_gallon_float'].min(), sa_drivers_df['cost_per_gallon_float'].max())
cmap = plt.get_cmap('viridis')  # A colormap without white space

# Plot the fuel prices over time for SA using a scatter plot with color mapping
plt.scatter(sa_drivers_df['date_fueled'], sa_drivers_df['cost_per_gallon_float'], 
            c=sa_drivers_df['cost_per_gallon_float'], cmap=cmap, norm=norm, marker='o')
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Price Over Time')
plt.colorbar(label='Price')  # Add a colorbar to show the price scale
plt.grid(True)
plt.show()


In [ ]:
# 4.3.3. Using a suitable plot, show if the difference in the number of people refueling on a Tuesday vs other days. [3]
# Using a bar graph, 2 bars mean of tuesdays other bar, mean of all other days
# Calculate the number of entries (rows) for each day of the week
sa_drivers_df['day_of_week'] = sa_drivers_df['date_captured'].dt.dayofweek
tuesday_count = sa_drivers_df[sa_drivers_df['day_of_week'] == 1].shape[0]
other_days_count = sa_drivers_df[sa_drivers_df['day_of_week'] != 1].shape[0]

# Total number of rows for each category
total_tuesdays = sa_drivers_df['day_of_week'].value_counts().get(1, 0)
total_other_days = sa_drivers_df['day_of_week'].value_counts().get(0, 0) + sa_drivers_df['day_of_week'].value_counts().get(2, 0) + \
                    sa_drivers_df['day_of_week'].value_counts().get(3, 0) + sa_drivers_df['day_of_week'].value_counts().get(4, 0) + \
                    sa_drivers_df['day_of_week'].value_counts().get(5, 0) + sa_drivers_df['day_of_week'].value_counts().get(6, 0)

# Calculate the averages
tuesday_avg = tuesday_count / (tuesday_count +   other_days_count)
other_days_avg = other_days_count / (tuesday_count +   other_days_count)

# Plot the averages
plt.figure(figsize=(8, 5))
plt.bar(['Tuesday', 'Other Days'], [tuesday_avg, other_days_avg], color=['blue', 'orange'])
plt.ylabel('Average Number of Refueling Instances')
plt.title('Average Refueling on Tuesdays vs Other Days')
plt.show()

In [ ]:
# 4.3.4. Now reduce your dataset to only the entries on the 1st Tuesday and 1st Wednesday in SA every month. [2]

def get_1st_tuesday_wednesday(df):
    result = []
    for month, group in df.groupby(df['date_fueled'].dt.to_period('M')):
        first_tuesday = group[(group['day_of_week'] == 1) & (group['date_fueled'].dt.day <= 7)]
        first_wednesday = group[(group['day_of_week'] == 2) & (group['date_fueled'].dt.day <= 7)]
        result.append(first_tuesday)
        result.append(first_wednesday)
    return pd.concat(result)

tues_weds_df = get_1st_tuesday_wednesday(sa_drivers_df)


In [ ]:
# 4.3.5. For each Tuesday and Wednesday, add an indicator for whether the price goes up or the price goes down that month
# So we make a new column

filtered_df = tues_weds_df.sort_values(by='date_fueled').reset_index(drop=True)
filtered_df['price_change'] = filtered_df['cost_per_gallon_float'].diff().fillna(0)
filtered_df['price_indicator'] = filtered_df['price_change'].apply(lambda x: 'Increase' if x > 0 else ('Decrease' if x < 0 else 'No Change'))

filtered_df.head(10)




In [ ]:
# 4.3.6 Do more people refuel on the first Wednesday of the month when the prices goes down? [2]
wednesday_price_down = filtered_df[(filtered_df['day_of_week'] == 2) & (filtered_df['price_indicator'] == 'Decrease')]
wednesday_avg_refuel = wednesday_price_down.shape[0]/filtered_df[(filtered_df['day_of_week'] == 2)].shape[0]
print(f"Average number of people refueling on the 1st Wednesday when the price goes down: {wednesday_avg_refuel:.2f}")

# 4.3.7. Do more people refuel on the first Tuesday of the month when the prices goes up? [2]
tuesday_price_up = filtered_df[(filtered_df['day_of_week'] == 1) & (filtered_df['price_indicator'] == 'Increase')]
tuesday_avg_refuel = tuesday_price_up.shape[0]/filtered_df[(filtered_df['day_of_week'] == 1)].shape[0]
print(f"Average number of people refueling on the 1st Tuesday when the price goes up: {tuesday_avg_refuel:.2f}")

In [ ]:
#hi